# Instituto Tecnologico de Costa Rica
# Ingenieria en Computacion
# Inteligencia artificial 
--- 
## Tarea Programada 3 
## Profesora : Maria Auxialiadora Mora
---
## Estudiantes: 
## Kenneth Corrales Lizano 2017151342
## Fabian Vives Castro 2018
---

In [16]:
#Bibliotecas
import pandas as pd
import numpy as np
import torch
import torch.nn as nn


#Bibliotecas Ejemplo 1 

#Bibliotecas Ejemplo 2
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import spacy

Hello


&nbsp;

# Ejercicio 1

### Describa el problema y el objetivo del ejercicio

### Describa los datos utilizados en el ejercicio

### Cargue, normalice y explore los datos


&nbsp;
# Ejercicio 2

### 1. 
Describa el problema y el objetivo del ejercicio.

A la hora de comprar un producto, ir a algun lugar, realizar alguna actividad..., basamos ya sea consiente o inconscientemente nuestras decisiones en base a opiniones externas, es por esto que muchas companias toman en especial cuidado el como lo ven sus consumidores ya que estos son los que posteriormente agregaran nueva clientela. Es por esto que es de suma importancia tener la calificacion y la informacion de los consumidores, para tomar decisiones al respecto de que esta bien, que es mejor cambiar asi como ideas de donde innovar. Ahora bien puesto que leer miles o millones de criticas redactadas en texto no es una tarea tan rapida, acudimos  a la tecnologia, en este caso en especifico a un algoritmo de procesamiento de lenguaje natural llamado analisis de sentimiento, en el que a groso modo, pretende clasificar cada critica  en clases definidas por el investigador, a fin de en una manera rapida, conocer los promedios de que tan satisfecho/insatisfecho se encuentran los usuarios de la plataforma tripadvisor en este caso.

### 2. Describa los datos utilizados en el ejercicio.

Como se esbozo anteriormente, se van a analizar una recopilacion de las criticas a los hoteles que se encuentran en la plataforma tripadvisor, este dataset consta de 2 columnas unicamente, la primera se refiere al detalle de la critica, es decir una breve descripcion de la misma, la segunda es un valor de estrellas del 1 al 5. La cantidad de filas es ligeramente superior a los 20 mil records.



### 3. Cargue y prepare los datos para ser introducidos a la red LSTM.

In [15]:
path= "tripadvisor_hotel_reviews.csv"
raw_data =pd.read_csv(path)

# podemos ver las dimensiones de la informacion
print(f"Tenemos {raw_data.shape[0]} filas  y {raw_data.shape[1]} columnas: el review y el rating")
# una visualizacion de las primeras filas de la data
raw_data.head(10)

Tenemos 20491 filas  y 2 columnas el review y el rating


,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
5,love monaco staff husband stayed hotel crazy w...,5
6,"cozy stay rainy city, husband spent 7 nights m...",5
7,"excellent staff, housekeeping quality hotel ch...",4
8,"hotel stayed hotel monaco cruise, rooms genero...",5
9,excellent stayed hotel monaco past w/e delight...,5


In [10]:
#Checamos si hay Nulos y los remplazamos con 
raw_data['Review'] = raw_data['Review'].fillna('')
raw_data['Rating'] = raw_data['Rating'].fillna('')

In [11]:

# Segun un chequeo general del documento parece que la data no tiene espacios nulos o vacios sin embargo, 
# puesto que son 20mil lineas lo mejor es que el sistema lo haga por nosotros
print("Cantidad de registros antes de la limpieza", raw_data.shape)
# iniciamos la variable de la data 
parsed_data = raw_data
# Creamos una variable con la informacion una vez parseada 
parsed_data['Review_Length'] = parsed_data['Review'].apply(lambda x: len(x.split()))
parsed_data = parsed_data[parsed_data['Review_Length'] != 0]
parsed_data = parsed_data[pd.notnull(parsed_data['Rating'])]
parsed_data = parsed_data[parsed_data['Rating'] != 0]
print("Cantidad de registros después de la limpieza", raw_data.shape)

Cantidad de registros antes de la limpieza (20491, 2)
Cantidad de registros después de la limpieza (20491, 3)


### 4. Estadisticas


In [12]:
parsed_data['Review_Length'] = parsed_data['Review'].apply(lambda x: len(x.split()))
parsed_data.head(10)

,Review,Rating,Review_Length
0,nice hotel expensive parking got good deal sta...,4,87
1,ok nothing special charge diamond member hilto...,2,250
2,nice rooms not 4* experience hotel monaco seat...,3,217
3,"unique, great stay, wonderful time hotel monac...",5,89
4,"great stay great stay, went seahawk game aweso...",5,191
5,love monaco staff husband stayed hotel crazy w...,5,134
6,"cozy stay rainy city, husband spent 7 nights m...",5,101
7,"excellent staff, housekeeping quality hotel ch...",4,85
8,"hotel stayed hotel monaco cruise, rooms genero...",5,59
9,excellent stayed hotel monaco past w/e delight...,5,35


In [13]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
parsed_data.describe()

,Rating,Review_Length
count,20491.00,20491.00
mean,3.95,104.38
std,1.23,100.66
min,1.00,7.00
25%,3.00,48.00
50%,4.00,77.00
75%,5.00,124.00
max,5.00,1931.00


In [14]:
# con esto podemos ordenar/ limpiar los datos para que sean mas facil trabajar 
parsed_data['Rating'].replace({1: 0, 2: 1, 3: 2, 4: 3, 5: 4}, inplace=True)
parsed_data.head(10)

,Review,Rating,Review_Length
0,nice hotel expensive parking got good deal sta...,3,87
1,ok nothing special charge diamond member hilto...,1,250
2,nice rooms not 4* experience hotel monaco seat...,2,217
3,"unique, great stay, wonderful time hotel monac...",4,89
4,"great stay great stay, went seahawk game aweso...",4,191
5,love monaco staff husband stayed hotel crazy w...,4,134
6,"cozy stay rainy city, husband spent 7 nights m...",4,101
7,"excellent staff, housekeeping quality hotel ch...",3,85
8,"hotel stayed hotel monaco cruise, rooms genero...",4,59
9,excellent stayed hotel monaco past w/e delight...,4,35


In [17]:
# Verificación de qué tan bien balanceadas están las clases. 
Counter(parsed_data['Rating'])

Counter({3: 6039, 1: 1793, 2: 2184, 4: 9054, 0: 1421})

### 5. Definicion de la red recurrente LSTM

